Final Code

In [ ]:
import requests
import pandas as pd
import time

def fetch_json_data(api_url):
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def fetch_until_target_time(start_time, target_time, symbol, interval, increment_time, batch_size=1000):
    """
    Fetch data from the API iteratively until the target time is reached.

    :param start_time: The initial start time in milliseconds.
    :param target_time: The target end time in milliseconds.
    :param symbol: The trading pair symbol (e.g., 'SOLUSDT').
    :param interval: The interval for the kline data (e.g., '1m').
    :param batch_size: Number of data points per batch (default: 1000).
    :return: A pandas DataFrame containing the aggregated data.
    """
    all_data = []
    current_time = start_time

    while current_time < target_time:
        api_url = f"https://www.binance.com/api/v3/uiKlines?endTime={current_time}&limit={batch_size}&symbol={symbol}&interval={interval}"
        batch_data = fetch_json_data(api_url)

        if batch_data:
            all_data.extend(batch_data)
        else:
            print("No data received or an error occurred.")
            break

        # Increment current_time by 1000 minutes (60,000 ms * 1000)
        current_time += increment_time
        # time.sleep(1)  # Respect API rate limits by pausing briefly

        # Break the loop if the next batch would exceed the target time
        if current_time >= target_time:
            print("Target time reached.")
            break

    # Convert aggregated data to a pandas DataFrame
    try:
        df = pd.DataFrame(all_data, columns=["OpenTime", "Open", "High", "Low", "Close", "Volume",
                                             "CloseTime", "QuoteAssetVolume", "NumberOfTrades",
                                             "TakerBuyBaseVolume", "TakerBuyQuoteVolume", "Ignore"])
        return df
    except ValueError as e:
        print(f"Error converting data to DataFrame: {e}")
        return pd.DataFrame()

# Example usage
if __name__ == "__main__":
    start_time = 1730745000000 #05 NOV 2024
    target_time = 1731609000000  # 15 NOV 2024

    # start_time = 1597912070407 #20 AUG 2024
    # target_time = 1731609000000  # 15 NOV 2024
    symbol = "SOLUSDT"
    # interval = "1m"
    interval = "1s"
    # increment_time = 1000 * 60 * 1000  # Increment by 1000 minutes (60,000 ms * 1000)
    increment_time = 1000 * 1000  # Increment by 1000 seconds (1000 ms * 1000)

    # Fetch and aggregate data until the target time
    aggregated_df = fetch_until_target_time(start_time, target_time, symbol, interval, increment_time)

    if not aggregated_df.empty:
        print("\nAggregated DataFrame:")
        print(aggregated_df.info())


Target time reached.

Aggregated DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864000 entries, 0 to 863999
Data columns (total 12 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   OpenTime             864000 non-null  int64 
 1   Open                 864000 non-null  object
 2   High                 864000 non-null  object
 3   Low                  864000 non-null  object
 4   Close                864000 non-null  object
 5   Volume               864000 non-null  object
 6   CloseTime            864000 non-null  int64 
 7   QuoteAssetVolume     864000 non-null  object
 8   NumberOfTrades       864000 non-null  int64 
 9   TakerBuyBaseVolume   864000 non-null  object
 10  TakerBuyQuoteVolume  864000 non-null  object
 11  Ignore               864000 non-null  object
dtypes: int64(3), object(9)
memory usage: 79.1+ MB
None


Testing Code


In [ ]:
# prompt: convert aggregated_df to csv

if not aggregated_df.empty:
    aggregated_df.to_csv('SOLUSD_1s_05NOV2024_to15NOV2024.csv', index=False)
    print("DataFrame saved to aggregated_data.csv")

DataFrame saved to aggregated_data.csv


In [ ]:
aggregated_df.tail()
# aggregated_df.to_json('aggregated_data.json', index=False)


,OpenTime,Open,High,Low,Close,Volume,CloseTime,QuoteAssetVolume,NumberOfTrades,TakerBuyBaseVolume,TakerBuyQuoteVolume,Ignore
1140995,1731608816000,215.71000000,215.71000000,215.71000000,215.71000000,0.00000000,1731608816999,0.00000000,0,0.00000000,0.00000000,0
1140996,1731608817000,215.71000000,215.71000000,215.71000000,215.71000000,0.00000000,1731608817999,0.00000000,0,0.00000000,0.00000000,0
1140997,1731608818000,215.71000000,215.71000000,215.70000000,215.70000000,6.33300000,1731608818999,1366.08412000,8,5.60200000,1208.40742000,0
1140998,1731608819000,215.71000000,215.71000000,215.70000000,215.70000000,0.60400000,1731608819999,130.28498000,4,0.21800000,47.02478000,0
1140999,1731608820000,215.70000000,215.70000000,215.70000000,215.70000000,0.76800000,1731608820999,165.65760000,1,0.00000000,0.00000000,0


In [ ]:
import requests
import pandas as pd

def fetch_json_data(api_url):
    try:
        # Sending a GET request to the API
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)

        # Parse the JSON data
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def display_data_with_pandas(json_data):
    try:
        # Convert JSON data to a pandas DataFrame
        df = pd.DataFrame(json_data)
        print("\nData displayed as a pandas DataFrame:")
        print(df)
    except ValueError as e:
        print(f"Error in converting JSON to DataFrame: {e}")

# Example usage
if __name__ == "__main__":
    # Replace this URL with any API endpoint that returns JSON data
    time = 1597912070407
    api_url = f"https://www.binance.com/api/v3/uiKlines?endTime={}&limit=1000&symbol=SOLUSDT&interval=1m"

    # Fetch JSON data
    json_data = fetch_json_data(api_url)

    if json_data:
        # Display JSON data using pandas
        display_data_with_pandas(json_data)



Data displayed as a pandas DataFrame:
                0           1           2           3           4   \
0    1597852080000  3.00000000  3.00090000  2.98930000  2.99770000   
1    1597852140000  2.98930000  2.99580000  2.98000000  2.99530000   
2    1597852200000  2.98230000  2.99290000  2.98000000  2.99290000   
3    1597852260000  2.98250000  3.00460000  2.98240000  3.00450000   
4    1597852320000  2.99820000  2.99820000  2.99800000  2.99800000   
..             ...         ...         ...         ...         ...   
995  1597911780000  2.99620000  2.99620000  2.99620000  2.99620000   
996  1597911840000  2.99620000  3.00000000  2.99610000  2.99610000   
997  1597911900000  2.99610000  3.00000000  2.99360000  2.99360000   
998  1597911960000  2.99370000  2.99370000  2.99110000  2.99110000   
999  1597912020000  2.99170000  2.99360000  2.99170000  2.99360000   

                5              6              7   8             9   \
0     771.96000000  1597852139999  2315.23058600  

Main Code File

In [ ]:
import requests
import pandas as pd
import time

def fetch_json_data(api_url):
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return None

def fetch_multiple_batches(start_time, symbol, interval, batch_size=1000, iterations=10):
    """
    Fetch multiple batches of data from the API, adding each batch to a pandas DataFrame.

    :param start_time: The initial start time in milliseconds.
    :param symbol: The trading pair symbol (e.g., 'SOLUSDT').
    :param interval: The interval for the kline data (e.g., '1m').
    :param batch_size: Number of data points per batch (default: 1000).
    :param iterations: Number of batches to fetch.
    :return: A pandas DataFrame containing the aggregated data.
    """
    all_data = []
    current_time = start_time

    for _ in range(iterations):
        api_url = f"https://www.binance.com/api/v3/uiKlines?endTime={current_time}&limit={batch_size}&symbol={symbol}&interval={interval}"
        batch_data = fetch_json_data(api_url)

        if batch_data:
            all_data.extend(batch_data)
        else:
            print("No data received or an error occurred.")
            break

        # Increment current_time by 1000 minutes (60,000 ms * 1000)
        current_time += 1000 * 60 * 1000
        # time.sleep(1)  # Respect API rate limits by pausing briefly

    # Convert aggregated data to a pandas DataFrame
    try:
        df = pd.DataFrame(all_data, columns=["OpenTime", "Open", "High", "Low", "Close", "Volume",
                                             "CloseTime", "QuoteAssetVolume", "NumberOfTrades",
                                             "TakerBuyBaseVolume", "TakerBuyQuoteVolume", "Ignore"])
        return df
    except ValueError as e:
        print(f"Error converting data to DataFrame: {e}")
        return pd.DataFrame()

# Example usage
if __name__ == "__main__":
    start_time = 1597912070407  # Replace with your desired start time in milliseconds
    symbol = "SOLUSDT"
    interval = "1m"

    # Fetch and aggregate multiple batches of data
    aggregated_df = fetch_multiple_batches(start_time, symbol, interval, iterations=1500)

    if not aggregated_df.empty:
        print("\nAggregated DataFrame:")
        print(aggregated_df.info())



Aggregated DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500000 entries, 0 to 1499999
Data columns (total 12 columns):
 #   Column               Non-Null Count    Dtype 
---  ------               --------------    ----- 
 0   OpenTime             1500000 non-null  int64 
 1   Open                 1500000 non-null  object
 2   High                 1500000 non-null  object
 3   Low                  1500000 non-null  object
 4   Close                1500000 non-null  object
 5   Volume               1500000 non-null  object
 6   CloseTime            1500000 non-null  int64 
 7   QuoteAssetVolume     1500000 non-null  object
 8   NumberOfTrades       1500000 non-null  int64 
 9   TakerBuyBaseVolume   1500000 non-null  object
 10  TakerBuyQuoteVolume  1500000 non-null  object
 11  Ignore               1500000 non-null  object
dtypes: int64(3), object(9)
memory usage: 137.3+ MB
None


In [ ]:
import requests
import pandas as pd
import io
import zipfile
from datetime import datetime, timedelta
import concurrent.futures

def download_and_process_file(date_str):
    """
    Download and process a single day's trading data
    """
    base_url = "https://data.binance.vision/data/spot/daily/trades/SOLUSDT/"
    filename = f"SOLUSDT-trades-{date_str}.zip"
    url = base_url + filename

    try:
        # Download the file
        response = requests.get(url)
        response.raise_for_status()

        # Read the zip file
        with zipfile.ZipFile(io.BytesIO(response.content)) as z:
            # Read CSV from zip
            with z.open(z.namelist()[0]) as f:
                df = pd.read_csv(f, names=['trade_id', 'price', 'quantity', 'quote_quantity',
                                         'timestamp', 'is_buyer_maker', 'is_best_match'])

        print(f"Successfully processed {date_str}")
        return df

    except Exception as e:
        print(f"Error processing {date_str}: {str(e)}")
        return None

def get_all_trading_data(start_date, end_date):
    """
    Download and combine trading data for a date range
    """
    # Convert dates to datetime objects
    start = datetime.strptime(start_date, '%Y-%m-%d')
    end = datetime.strptime(end_date, '%Y-%m-%d')

    # Generate list of dates
    dates = []
    current = start
    while current <= end:
        dates.append(current.strftime('%Y-%m-%d'))
        current += timedelta(days=1)

    # Download and process files in parallel
    all_data = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(download_and_process_file, date): date
                  for date in dates}

        for future in concurrent.futures.as_completed(futures):
            df = future.result()
            if df is not None:
                all_data.append(df)

    # Combine all dataframes
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)

        # Convert timestamp to datetime
        combined_df['timestamp'] = pd.to_datetime(combined_df['timestamp'], unit='ms')

        # Sort by timestamp
        combined_df = combined_df.sort_values('timestamp')

        return combined_df
    else:
        return None

def analyze_trading_data(df):
    """
    Perform basic analysis on the trading data
    """
    if df is None or len(df) == 0:
        return "No data to analyze"

    analysis = {
        'total_trades': len(df),
        'start_time': df['timestamp'].min(),
        'end_time': df['timestamp'].max(),
        'average_price': df['price'].mean(),
        'min_price': df['price'].min(),
        'max_price': df['price'].max(),
        'total_volume': df['quantity'].sum(),
        'total_quote_volume': df['quote_quantity'].sum(),
        'buyer_maker_percentage': (df['is_buyer_maker'].mean() * 100)
    }

    return analysis

# Example usage
if __name__ == "__main__":
    # Example date range (adjust as needed)
    start_date = '2024-11-05'  # From your earliest date in the image
    end_date = '2024-11-12'    # To your latest date in the image

    print("Downloading and processing data...")
    df = get_all_trading_data(start_date, end_date)

    if df is not None:
        print("\nData downloaded successfully!")
        print(f"Total rows: {len(df)}")

        print("\nBasic Analysis:")
        analysis = analyze_trading_data(df)
        for key, value in analysis.items():
            print(f"{key}: {value}")

        # Save to file if needed
        df.to_csv('combined_SOLUSDT_trades.csv', index=False)
        print("\nData saved to 'combined_SOLUSDT_trades.csv'")

Successfully processed 2024-11-09
Successfully processed 2024-11-05
Successfully processed 2024-11-07
Successfully processed 2024-11-08
Successfully processed 2024-11-06
Successfully processed 2024-11-10
Successfully processed 2024-11-11
Successfully processed 2024-11-12

Data downloaded successfully!
Total rows: 24760362

Basic Analysis:
total_trades: 24760362
start_time: 2024-11-05 00:00:00.177000
end_time: 2024-11-12 23:59:59.083000
average_price: 201.8833584008187
min_price: 156.86
max_price: 225.21
total_volume: 51902197.72399988
total_quote_volume: 10375688014.011381
buyer_maker_percentage: 49.52037050185292

Data saved to 'combined_SOLUSDT_trades.csv'


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24760362 entries, 1495027 to 24760361
Data columns (total 7 columns):
 #   Column          Dtype         
---  ------          -----         
 0   trade_id        int64         
 1   price           float64       
 2   quantity        float64       
 3   quote_quantity  float64       
 4   timestamp       datetime64[ns]
 5   is_buyer_maker  bool          
 6   is_best_match   bool          
dtypes: bool(2), datetime64[ns](1), float64(3), int64(1)
memory usage: 1.2 GB
